## Build history of commodity COTs

1. Retrieve data from the CFTC website (www.cft.gov/files);
2. Extract data from the most important Commercial and Non Commercial long and short columns;
3. Create "net" columns for each important category;
4. Merge this data with the ETF history data created in the previous steps


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [35]:
import pandas as pd
import numpy as np
import os, sys,glob
import datetime
%matplotlib inline
import matplotlib.pyplot as plt
# import plotly.plotly as py
import plotly.graph_objs as go
import zipfile
import urllib.request
from PIL import Image
def str_to_date(d):
    try:
        dt = datetime.datetime.strptime(str(d),'%Y-%m-%d')
    except:
        return None
    return dt

# Make important folders
TEMP_FOLDER = './temp_folder'
try:
    os.mkdir(TEMP_FOLDER)
except:
    pass
try:
    os.mkdir(f'{TEMP_FOLDER}/cot')
except:
    pass
try:
    os.mkdir(f'{TEMP_FOLDER}/zip')
except:
    pass


___
## First, decide if you want to re-create the ETF and COT data, or just retrieve the previously saved data DataFrames.

In [36]:
CREATE_COT_DATA = True
cot_save_path = './cot_new_history.csv'
cot_net_save_path = './cot_net_new_history.csv'
YEAR_OFFSET = 0 if datetime.datetime.now() > datetime.datetime(2020,1,10) else 1

In [37]:
YEAR_OFFSET

0

### Main column names

In [38]:
basic_cols = ['Market_and_Exchange_Names','As_of_Date_In_Form_YYMMDD','Open_Interest_All']
long_cols = basic_cols + ['Prod_Merc_Positions_Long_All','Swap_Positions_Long_All',
            'M_Money_Positions_Long_All','Other_Rept_Positions_Long_All',
            'NonRept_Positions_Long_All','Tot_Rept_Positions_Long_All']
short_cols = basic_cols + ['Prod_Merc_Positions_Short_All','Swap_Positions_Short_All',
            'M_Money_Positions_Short_All','Other_Rept_Positions_Short_All',
            'NonRept_Positions_Short_All','Tot_Rept_Positions_Short_All']
summary_types = ['prod','swap','monman','other','nonrep','totrep']
summary_cols_dict = {summary_types[i]:(long_cols[i+len(basic_cols)],short_cols[i+len(basic_cols)]) for i in range(len(summary_types))}


In [39]:
summary_cols_dict

{'prod': ('Prod_Merc_Positions_Long_All', 'Prod_Merc_Positions_Short_All'),
 'swap': ('Swap_Positions_Long_All', 'Swap_Positions_Short_All'),
 'monman': ('M_Money_Positions_Long_All', 'M_Money_Positions_Short_All'),
 'other': ('Other_Rept_Positions_Long_All', 'Other_Rept_Positions_Short_All'),
 'nonrep': ('NonRept_Positions_Long_All', 'NonRept_Positions_Short_All'),
 'totrep': ('Tot_Rept_Positions_Long_All', 'Tot_Rept_Positions_Short_All')}

___
## Process CFTC COT Data
___

### Initial processing
1. Download zip files from www.cft.gov/files;
2. Unip the files using the zipfile package;
3. Read each csv (usually named Annual.TXT), and merge them into the df_cot DataFrame.


In [40]:
zip_download_folder = f'{TEMP_FOLDER}/zip'

In [41]:
f'{zip_download_folder}/*.csv'

'./temp_folder/zip/*.csv'

In [42]:
glob.glob(f'{zip_download_folder}/*')

['./temp_folder/zip/Annual.TXT',
 './temp_folder/zip/annual_2007.txt',
 './temp_folder/zip/dea_fut_xls_2000.0.zip',
 './temp_folder/zip/dea_fut_xls_2001.0.zip',
 './temp_folder/zip/dea_fut_xls_2002.0.zip',
 './temp_folder/zip/dea_fut_xls_2003.0.zip',
 './temp_folder/zip/dea_fut_xls_2004.0.zip',
 './temp_folder/zip/dea_fut_xls_2005.0.zip',
 './temp_folder/zip/dea_fut_xls_2006.0.zip',
 './temp_folder/zip/dea_fut_xls_2007.0.zip',
 './temp_folder/zip/dea_fut_xls_2008.0.zip',
 './temp_folder/zip/dea_fut_xls_2009.0.zip',
 './temp_folder/zip/dea_fut_xls_2010.0.zip',
 './temp_folder/zip/dea_fut_xls_2011.0.zip',
 './temp_folder/zip/dea_fut_xls_2012.0.zip',
 './temp_folder/zip/dea_fut_xls_2013.0.zip',
 './temp_folder/zip/dea_fut_xls_2014.0.zip',
 './temp_folder/zip/dea_fut_xls_2015.0.zip',
 './temp_folder/zip/dea_fut_xls_2016.0.zip',
 './temp_folder/zip/dea_fut_xls_2017.0.zip',
 './temp_folder/zip/dea_fut_xls_2018.0.zip',
 './temp_folder/zip/dea_fut_xls_2019.0.zip',
 './temp_folder/zip/dea_fut_x

In [43]:
if CREATE_COT_DATA:
    last_year = datetime.datetime.now().year
    year_list = np.arange(2010,last_year+1)
    zip_download_folder = f'{TEMP_FOLDER}/zip'
    df_cot_temp = None
    df_cot = None
    for y in year_list:
        yint = int(y)
#         url = f"https://www.cftc.gov/files/dea/history/deacot{yint}.zip" 
        url = f'https://www.cftc.gov/files/dea/history/fut_disagg_txt_{yint}.zip'
        path_to_zip_file = f'{zip_download_folder}/fut_disagg_txt_{y}.zip'
        if len(glob.glob(f'{zip_download_folder}/fut_disagg_txt_{y}.csv'))>0:
            print(f'Already downloaded fut_disagg_txt_{y}.csv')
            continue
        print(f'About to downloaded fut_disagg_txt_{y}.csv')


About to downloaded fut_disagg_txt_2010.csv
About to downloaded fut_disagg_txt_2011.csv
About to downloaded fut_disagg_txt_2012.csv
About to downloaded fut_disagg_txt_2013.csv
About to downloaded fut_disagg_txt_2014.csv
About to downloaded fut_disagg_txt_2015.csv
About to downloaded fut_disagg_txt_2016.csv
About to downloaded fut_disagg_txt_2017.csv
About to downloaded fut_disagg_txt_2018.csv
About to downloaded fut_disagg_txt_2019.csv
About to downloaded fut_disagg_txt_2020.csv


In [44]:
if CREATE_COT_DATA:
    last_year = datetime.datetime.now().year - YEAR_OFFSET
    year_list = np.arange(2010,last_year+1)
    zip_download_folder = f'{TEMP_FOLDER}/zip'
    df_cot_temp = None
    df_cot = None
    for y in year_list:
        yint = int(y)
#         url = f"https://www.cftc.gov/files/dea/history/deacot{yint}.zip" 
        url = f'https://www.cftc.gov/files/dea/history/fut_disagg_txt_{yint}.zip'
        path_to_zip_file = f'{zip_download_folder}/fut_disagg_txt_{y}.zip'
        if len(glob.glob(f'{zip_download_folder}/fut_disagg_txt_{y}.csv'))>0:
            print(f'Already downloaded fut_disagg_txt_{y}.csv')
            continue
        if not os.path.isfile(path_to_zip_file) or y >= last_year:
            print(f'retrieving cot zip file from {url}')
            try:
                urllib.request.urlretrieve(url, path_to_zip_file)    
            except:
                import time
                time.sleep(1)
                urllib.request.urlretrieve(url, path_to_zip_file)    
        zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
        zip_ref.extractall(zip_download_folder)
        zip_ref.close()
#         df_cot_temp = pd.read_csv(f'{zip_download_folder}/Annual.TXT')
        df_cot_temp = pd.read_csv(f'{zip_download_folder}/f_year.txt')
        if df_cot is None:
            df_cot = df_cot_temp.copy()
        else:
            df_cot = df_cot.append(df_cot_temp,ignore_index=True)
            df_cot.index = list(range(len(df_cot)))
        print(f'processed cot csv file from {url}. Length = {len(df_cot_temp)}')
        
df_cot.head()

processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2010.zip. Length = 5547
processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2011.zip. Length = 5486
processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2012.zip. Length = 5938
processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2013.zip. Length = 8270
processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2014.zip. Length = 10271
processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2015.zip. Length = 9762
processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2016.zip. Length = 10520
processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2017.zip. Length = 10204
processed cot csv file from https://www.cftc.gov/files/dea/history/fut_disagg_txt_2018.zip. Length = 10874
processed cot csv file from https://www.cf

,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,Report_Date_as_MM_DD_YYYY,CFTC_Contract_Market_Code,CFTC_Market_Code,CFTC_Region_Code,CFTC_Commodity_Code,Open_Interest_All,Prod_Merc_Positions_Long_All,Prod_Merc_Positions_Short_All,...,Conc_Net_LE_4_TDR_Short_Other,Conc_Net_LE_8_TDR_Long_Other,Conc_Net_LE_8_TDR_Short_Other,Contract_Units,CFTC_Contract_Market_Code_Quotes,CFTC_Market_Code_Quotes,CFTC_Commodity_Code_Quotes,CFTC_SubGroup_Code,FutOnly_or_Combined,Report_Date_as_YYYY-MM-DD
0,WHEAT - CHICAGO BOARD OF TRADE,101228,2010-12-28,001602,CBT,0,1,488334,49996,245335,...,16.4,41.3,22.5,"(CONTRACTS OF 5,000 BUSHELS)",001602,CBT,1,A10,FutOnly,NaN
1,WHEAT - CHICAGO BOARD OF TRADE,101221,2010-12-21,001602,CBT,0,1,480241,51194,237590,...,16.4,41.3,22.7,"(CONTRACTS OF 5,000 BUSHELS)",001602,CBT,1,A10,FutOnly,NaN
2,WHEAT - CHICAGO BOARD OF TRADE,101214,2010-12-14,001602,CBT,0,1,478364,52201,235461,...,16.5,41.2,23.0,"(CONTRACTS OF 5,000 BUSHELS)",001602,CBT,1,A10,FutOnly,NaN
3,WHEAT - CHICAGO BOARD OF TRADE,101207,2010-12-07,001602,CBT,0,1,479187,53882,229541,...,16.4,42.3,22.7,"(CONTRACTS OF 5,000 BUSHELS)",001602,CBT,1,A10,FutOnly,NaN
4,WHEAT - CHICAGO BOARD OF TRADE,101130,2010-11-30,001602,CBT,0,1,459421,60038,198370,...,16.6,46.5,22.7,"(CONTRACTS OF 5,000 BUSHELS)",001602,CBT,1,A10,FutOnly,NaN


___
### Make column names easier to process, make main date field a datetime object, and sort the DataFrame
___

___
### Show important columns for a specific  commodity
___

In [45]:
if CREATE_COT_DATA:
    col_rename_dict = {c:c.strip().replace('__','_').replace(' ','_').replace('-','_').replace('(','').replace(')','') for c in df_cot.columns.values}
    df_cot2 = df_cot.rename(columns=col_rename_dict)
    df_cot2 = df_cot2.drop(columns=['Report_Date_as_MM_DD_YYYY'])
    df_cot2.Market_and_Exchange_Names = df_cot2.Market_and_Exchange_Names.str.strip()
    l = lambda s:datetime.datetime(2000+int(str(s)[0:2]),int(str(s)[2:4]),int(str(s)[4:6]))
    df_cot2.As_of_Date_In_Form_YYMMDD = df_cot2.As_of_Date_In_Form_YYMMDD.apply(l)
    df_cot2 = df_cot2.sort_values(['Market_and_Exchange_Names','As_of_Date_In_Form_YYMMDD'])
    df_cot2.to_csv(cot_save_path,index=False)
    
    

In [54]:
df_cot2 = pd.read_csv(cot_save_path)
df_cot2.As_of_Date_In_Form_YYMMDD = df_cot2.As_of_Date_In_Form_YYMMDD.apply(str_to_date)
cot_beg_date = datetime.datetime.now() - datetime.timedelta(2000)
cot_beg_date = df_cot2.As_of_Date_In_Form_YYMMDD.min()
# df_commod = df_cot2[df_cot2.Market_and_Exchange_Names.str.contains(commod)][df_cot2.As_of_Date_In_Form_YYMMDD>=cot_beg_date]
df_commod = df_cot2[df_cot2.As_of_Date_In_Form_YYMMDD>=cot_beg_date]
df_commod_basic = df_commod[basic_cols]
df_commod_long = df_commod[long_cols]
df_commod_short = df_commod[short_cols]


In [55]:
df_cot2.As_of_Date_In_Form_YYMMDD.min(),df_commod_long.As_of_Date_In_Form_YYMMDD.min()

(Timestamp('2010-01-05 00:00:00'), Timestamp('2010-01-05 00:00:00'))

### Show basic open interest info

In [56]:
commod = 'CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE EXCHANGE'
[c for c in df_commod.Market_and_Exchange_Names.values if 'CRUDE' in c]

['BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK MERCANTILE EXCHANGE',
 'BRENT CRUDE OIL LAST DAY - NEW YORK ME

In [57]:
df_commod_basic[df_cot2.Market_and_Exchange_Names.str.contains(commod)].sort_values('As_of_Date_In_Form_YYMMDD',ascending=False).head()


,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,Open_Interest_All
15260,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-06-09,2126629
15259,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-06-02,2127664
15258,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-05-26,2135678
15257,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-05-19,2158749
15256,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-05-12,2248020


### Show important "long" position info

In [58]:
df_commod_long[df_cot2.Market_and_Exchange_Names.str.contains(commod)].sort_values('As_of_Date_In_Form_YYMMDD',ascending=False).head()

,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,Open_Interest_All,Prod_Merc_Positions_Long_All,Swap_Positions_Long_All,M_Money_Positions_Long_All,Other_Rept_Positions_Long_All,NonRept_Positions_Long_All,Tot_Rept_Positions_Long_All
15260,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-06-09,2126629,397123,153372,444681,288253,104697,2021932
15259,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-06-02,2127664,400183,148732,438274,290898,103180,2024484
15258,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-05-26,2135678,408856,141849,420655,280579,101226,2034452
15257,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-05-19,2158749,425213,139073,401853,290164,103753,2054996
15256,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-05-12,2248020,491543,134875,405071,304486,102997,2145023


### Show important "short" position info

In [59]:
df_commod_short[df_cot2.Market_and_Exchange_Names.str.contains(commod)].sort_values('As_of_Date_In_Form_YYMMDD',ascending=False).head()

,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,Open_Interest_All,Prod_Merc_Positions_Short_All,Swap_Positions_Short_All,M_Money_Positions_Short_All,Other_Rept_Positions_Short_All,NonRept_Positions_Short_All,Tot_Rept_Positions_Short_All
15260,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-06-09,2126629,574214,584844,47132,117893,64043,2062586
15259,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-06-02,2127664,570274,578655,44143,116699,71496,2056168
15258,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-05-26,2135678,547988,573857,45845,112815,72660,2063018
15257,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-05-19,2158749,546228,593013,41670,106744,72401,2086348
15256,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-05-12,2248020,600425,583964,53386,115152,86045,2161975


### CREATE important "net" position info for other anlysis notebooks

In [60]:
df_commod_long.head()

,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,Open_Interest_All,Prod_Merc_Positions_Long_All,Swap_Positions_Long_All,M_Money_Positions_Long_All,Other_Rept_Positions_Long_All,NonRept_Positions_Long_All,Tot_Rept_Positions_Long_All
0,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2013-11-12,308501,123580,51667,25819,14403,34785,273716
1,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2013-11-19,307976,120906,48750,26373,14699,37463,270513
2,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2013-11-26,291802,100428,48332,31971,12962,41124,250678
3,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2013-12-03,289308,91532,49249,37253,13249,43056,246252
4,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2013-12-10,305984,102166,49150,42992,12938,41888,264096


In [61]:
df_commod_long.head()

,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,Open_Interest_All,Prod_Merc_Positions_Long_All,Swap_Positions_Long_All,M_Money_Positions_Long_All,Other_Rept_Positions_Long_All,NonRept_Positions_Long_All,Tot_Rept_Positions_Long_All
0,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2013-11-12,308501,123580,51667,25819,14403,34785,273716
1,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2013-11-19,307976,120906,48750,26373,14699,37463,270513
2,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2013-11-26,291802,100428,48332,31971,12962,41124,250678
3,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2013-12-03,289308,91532,49249,37253,13249,43056,246252
4,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2013-12-10,305984,102166,49150,42992,12938,41888,264096


In [62]:
df_commod_short.head()

,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,Open_Interest_All,Prod_Merc_Positions_Short_All,Swap_Positions_Short_All,M_Money_Positions_Short_All,Other_Rept_Positions_Short_All,NonRept_Positions_Short_All,Tot_Rept_Positions_Short_All
0,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2013-11-12,308501,134742,8652,33006,42416,31438,277063
1,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2013-11-19,307976,132725,10419,31557,43331,30159,277817
2,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2013-11-26,291802,127973,11621,21389,46914,26920,264882
3,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2013-12-03,289308,134648,12283,17855,44871,24682,264626
4,"#2 HEATING OIL, NY HARBOR-ULSD - NEW YORK MERC...",2013-12-10,305984,146938,11548,17499,49442,23707,282277


In [63]:
df_commod_net = df_commod_long.merge(df_commod_short,how='inner',on=basic_cols)
print(len(df_commod_net),len(df_commod_long),len(df_commod_short))


92377 92377 92377


In [64]:
df_commod_net = df_commod_long.merge(df_commod_short,how='inner',on=basic_cols)
print(len(df_commod_net),len(df_commod_long),len(df_commod_short))

net_cols = [c1 for c1 in long_cols + short_cols if c1 not in basic_cols]
for c2 in net_cols:
    df_commod_net = df_commod_net[df_commod_net[c2].astype(str).str.contains('[0-9]')]
print(len(df_commod_net))


for p in summary_cols_dict.keys():
    t = summary_cols_dict[p]
    lc = t[0]
    sc = t[1]
    df_commod_net[p+'_net'] = df_commod_net[lc].astype(float) - df_commod_net[sc].astype(float)
    df_commod_net[p+'_ratio'] = df_commod_net[lc].astype(float) / df_commod_net[sc].astype(float)
print(len(df_commod_net))

sort_cols = ['Market_and_Exchange_Names','As_of_Date_In_Form_YYMMDD']
df_commod_net = df_commod_net.sort_values(sort_cols)


92377 92377 92377
92377
92377


In [65]:
ratio_cols = ['prod_ratio','monman_ratio','swap_ratio','other_ratio','nonrep_ratio']
df_commod_net[df_commod_net.Market_and_Exchange_Names.str.contains(commod)][sort_cols + ratio_cols].sort_values(sort_cols,ascending=False).head()



,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,prod_ratio,monman_ratio,swap_ratio,other_ratio,nonrep_ratio
15260,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-06-09,0.691594,9.434800,0.262244,2.445039,1.634792
15259,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-06-02,0.701738,9.928505,0.257031,2.492721,1.443158
15258,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-05-26,0.746104,9.175592,0.247185,2.487072,1.393146
15257,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-05-19,0.778453,9.643701,0.234519,2.718317,1.433033
15256,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE E...",2020-05-12,0.818658,7.587589,0.230965,2.644209,1.197013


In [66]:
nan_cols = df_commod_net.columns[df_commod_net.isna().any()].tolist()
if len(nan_cols)>0:
    print(nan_cols)
    df_commod_net = df_commod_net.fillna(0)
    print(df_commod_net[df_commod_net.monman_ratio.isnull()][ratio_cols])

['swap_ratio', 'monman_ratio', 'other_ratio', 'nonrep_ratio']
Empty DataFrame
Columns: [prod_ratio, monman_ratio, swap_ratio, other_ratio, nonrep_ratio]
Index: []


In [67]:
df_commod_net.columns.values

array(['Market_and_Exchange_Names', 'As_of_Date_In_Form_YYMMDD',
       'Open_Interest_All', 'Prod_Merc_Positions_Long_All',
       'Swap_Positions_Long_All', 'M_Money_Positions_Long_All',
       'Other_Rept_Positions_Long_All', 'NonRept_Positions_Long_All',
       'Tot_Rept_Positions_Long_All', 'Prod_Merc_Positions_Short_All',
       'Swap_Positions_Short_All', 'M_Money_Positions_Short_All',
       'Other_Rept_Positions_Short_All', 'NonRept_Positions_Short_All',
       'Tot_Rept_Positions_Short_All', 'prod_net', 'prod_ratio',
       'swap_net', 'swap_ratio', 'monman_net', 'monman_ratio',
       'other_net', 'other_ratio', 'nonrep_net', 'nonrep_ratio',
       'totrep_net', 'totrep_ratio'], dtype=object)

In [68]:
print(f'saving {len(df_commod_net)} records')
df_commod_net.to_csv(cot_net_save_path,index=False)

saving 92377 records


## END

In [69]:
len(df_commod_net[df_commod_net.Market_and_Exchange_Names.str.contains('SILVER - COMMODITY EXCHANGE INC.')])

544

In [70]:
df_cot2[df_cot2.Market_and_Exchange_Names.str.lower().str.contains('10')].Market_and_Exchange_Names.unique()


array(['1000 CALIFORNIA CARBON ALLOWANCE - ICE FUTURES ENERGY DIV',
       'HENRY HUB NAT GAS FINL-10000 - NASDAQ FUTURES',
       'HHUB NAT GAS PENULT FINL-10000 - NASDAQ FUTURES',
       'NFX SM10TC SUPRAMAX 10T/C AVG - NASDAQ FUTURES'], dtype=object)

In [71]:
list(filter(lambda s:'As_of_Date_In_Form_YYMMDD' in s,df_cot2.columns.values))

['As_of_Date_In_Form_YYMMDD']

In [72]:
df_wheat = df_cot2[df_cot2.Market_and_Exchange_Names.str.contains('WHEAT-SRW - CHICAGO BOARD OF TRADE')]
len(df_wheat)
# np.sort(df_wheat.As_of_Date_In_Form_YYMMDD.unique())

338

In [73]:
df_cot2[df_cot2.Market_and_Exchange_Names.str.contains('WHEAT-HRW - CHICAGO BOARD OF TRADE')].As_of_Date_In_Form_YYMMDD.max()


Timestamp('2020-06-09 00:00:00')

In [74]:
df_cot2.As_of_Date_In_Form_YYMMDD.min()

Timestamp('2010-01-05 00:00:00')

In [75]:
df_commod_net.As_of_Date_In_Form_YYMMDD.min()

Timestamp('2010-01-05 00:00:00')